# 사진_유사도_비교_서비스_mediapipe_안면인식_인증

In [ ]:
!pip install -q mediapipe

In [ ]:
!wget -O embedder.tflite -q https://storage.googleapis.com/mediapipe-models/image_embedder/mobilenet_v3_small/float32/1/mobilenet_v3_small.tflite

In [ ]:
import urllib
import cv2
import matplotlib.pyplot as plt

IMAGE_FILENAMES = ['burger.jpg', 'burger_crop.jpg']

for name in IMAGE_FILENAMES:
    url = f'https://storage.googleapis.com/mediapipe-assets/{name}'
    urllib.request.urlretrieve(url, name)

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

def resize_and_show(image):
    h, w = image.shape[:2]
    if h < w:
        img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
    else:
        img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
    return img

# 이미지 미리보기
images = {name: cv2.imread(name) for name in IMAGE_FILENAMES}
for name, image in images.items():
    print(name)
    resized_image = resize_and_show(image)
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.title(name)
    plt.axis('off')
    plt.show()


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Create options for Image Embedder
base_options = python.BaseOptions(model_asset_path='embedder.tflite')
l2_normalize = True #@param {type:"boolean"}
quantize = True #@param {type:"boolean"}
options = vision.ImageEmbedderOptions(
    base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)


# Create Image Embedder
with vision.ImageEmbedder.create_from_options(options) as embedder:

    # Format images for MediaPipe
    first_image = mp.Image.create_from_file(IMAGE_FILENAMES[0])
    second_image = mp.Image.create_from_file(IMAGE_FILENAMES[1])
    first_embedding_result = embedder.embed(first_image)
    second_embedding_result = embedder.embed(second_image)

    # Calculate and print similarity
    similarity = vision.ImageEmbedder.cosine_similarity(
      first_embedding_result.embeddings[0],
      second_embedding_result.embeddings[0])
    print(similarity)

In [ ]:
import gradio as gr
import urllib
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 이미지 임베딩 및 유사도 계산 함수
def image_to_text(image1, image2):
    # 이미지 파일 저장
    cv2.imwrite("image1.jpg", image1)
    cv2.imwrite("image2.jpg", image2)
    
    # Create options for Image Embedder
    base_options = python.BaseOptions(model_asset_path='./mobilenet_v3_large.tflite')
    l2_normalize = True
    quantize = True
    options = vision.ImageEmbedderOptions(
        base_options=base_options, l2_normalize=l2_normalize, quantize=quantize)

    # Create Image Embedder
    with vision.ImageEmbedder.create_from_options(options) as embedder:
        # Format images for MediaPipe
        first_image = mp.Image.create_from_file("image1.jpg")
        second_image = mp.Image.create_from_file("image2.jpg")
        first_embedding_result = embedder.embed(first_image)
        second_embedding_result = embedder.embed(second_image)

        # Calculate and print similarity
        similarity = vision.ImageEmbedder.cosine_similarity(
            first_embedding_result.embeddings[0],
            second_embedding_result.embeddings[0])
        return f"Similarity: {similarity:.4f}"

# Gradio 인터페이스 생성
with gr.Blocks() as app:
    gr.Markdown("# Image Similarity Comparison")
    
    with gr.Tab("Image Upload"):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="First Image")
            with gr.Column():
                image2 = gr.Image(label="Second Image")
        output = gr.Textbox(label="Similarity Output")
        convert_btn = gr.Button("Compare")
        convert_btn.click(
            fn=image_to_text, inputs=[image1, image2], outputs=output
        )

app.launch(inline=False, share=True)


# Deepface

In [ ]:
# !pip install deepface

In [ ]:
# !pip install tf-keras

In [ ]:
# !pip install --upgrade deepface

In [ ]:
import gradio as gr
from deepface import DeepFace
import cv2

# 얼굴 임베딩 및 유사도 계산 함수
def image_to_text(image1, image2):
    # 이미지 파일 저장
    cv2.imwrite("image1.jpg", image1)
    cv2.imwrite("image2.jpg", image2)
    
    # VGG-Face 모델을 사용하여 얼굴 유사도 계산
    result = DeepFace.verify("image1.jpg", "image2.jpg", model_name='VGG-Face')
    
    # 거리 값을 기준으로 유사도 출력
    similarity = result['distance']
    return f"Similarity: {similarity:.4f}"

# Gradio 인터페이스 생성
with gr.Blocks() as app:
    gr.Markdown("# Face Similarity Comparison using DeepFace")
    
    with gr.Tab("Image Upload"):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="First Image")
            with gr.Column():
                image2 = gr.Image(label="Second Image")
        output = gr.Textbox(label="Similarity Output")
        convert_btn = gr.Button("Compare")
        convert_btn.click(
            fn=image_to_text, inputs=[image1, image2], outputs=output
        )

app.launch(inline=False, share=True)


In [ ]:
import gradio as gr
from deepface import DeepFace
import cv2

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
  "GhostFaceNet",
]

metrics = ["cosine", "euclidean", "euclidean_l2"]

# 얼굴 임베딩 및 유사도 계산 함수
def image_to_text(image1, image2, model, metric):
    # 이미지 파일 저장
    cv2.imwrite("image1.jpg", image1)
    cv2.imwrite("image2.jpg", image2)
    
    # 얼굴 임베딩 생성
    embedding_objs1 = DeepFace.represent(img_path="image1.jpg", model_name=model)
    embedding_objs2 = DeepFace.represent(img_path="image2.jpg", model_name=model)
    
    # 임베딩 벡터 추출
    embedding1 = embedding_objs1[0]["embedding"]
    embedding2 = embedding_objs2[0]["embedding"]
    
    # 유사도 계산
    result = DeepFace.verify(
        img1_path="image1.jpg", 
        img2_path="image2.jpg", 
        model_name=model,
        distance_metric=metric
    )
    similarity = result['distance']
    return f"Similarity: {similarity:.4f}"

# Gradio 인터페이스 생성
with gr.Blocks() as app:
    gr.Markdown("# Face Similarity Comparison using DeepFace")
    
    with gr.Tab("Image Upload"):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="First Image")
            with gr.Column():
                image2 = gr.Image(label="Second Image")
        model = gr.Dropdown(label="Model", choices=models, value="VGG-Face")
        metric = gr.Dropdown(label="Distance Metric", choices=metrics, value="cosine")
        output = gr.Textbox(label="Similarity Output")
        convert_btn = gr.Button("Compare")
        convert_btn.click(
            fn=image_to_text, inputs=[image1, image2, model, metric], outputs=output
        )

app.launch(inline=False, share=True)


In [1]:
import gradio as gr
from deepface import DeepFace
import cv2
from scipy.spatial import distance

# 얼굴 임베딩 및 유사도 계산 함수
def image_to_text(image1, image2):
    # 이미지 파일 저장
    cv2.imwrite("image1.jpg", image1)
    cv2.imwrite("image2.jpg", image2)
    
    # VGG-Face 모델을 사용하여 임베딩 생성
    embedding_objs1 = DeepFace.represent(img_path="image1.jpg", model_name="VGG-Face")
    embedding_objs2 = DeepFace.represent(img_path="image2.jpg", model_name="VGG-Face")
    
    # 임베딩 추출
    embedding1 = embedding_objs1[0]["embedding"]
    embedding2 = embedding_objs2[0]["embedding"]
    
    # 코사인 유사도를 계산
    cosine_similarity = 1 - distance.cosine(embedding1, embedding2)
    
    return f"Cosine Similarity: {cosine_similarity:.4f}"

# Gradio 인터페이스 생성
with gr.Blocks() as app:
    gr.Markdown("# Face Similarity Comparison using DeepFace (Embeddings)")
    
    with gr.Tab("Image Upload"):
        with gr.Row():
            with gr.Column():
                image1 = gr.Image(label="First Image")
            with gr.Column():
                image2 = gr.Image(label="Second Image")
        output = gr.Textbox(label="Similarity Output")
        convert_btn = gr.Button("Compare")
        convert_btn.click(
            fn=image_to_text, inputs=[image1, image2], outputs=output
        )

app.launch(inline=False, share=True)


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
2024-09-24 15:03:56.689343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: At

ImportError: cannot import name 'build_model' from partially initialized module 'deepface.modules.modeling' (most likely due to a circular import) (/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/deepface/modules/modeling.py)